In [ ]:
from_drive=True

In [ ]:
## GIT ON COLAB ONLY
try:
    from google.colab import userdata
    git_token = userdata.get('gitToken')
    git_user = userdata.get('gitUser')
    git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
    branch_to_pull = 'dev'
    !git clone {git_url}
    %cd Atlas
    !git checkout -b colab_branch
    !git pull origin {branch_to_pull}
    !pip install -r PipelinesConsumo/src/requirements.txt
    %cd PipelinesConsumo
except Exception as e:
    print(e)
    print('Running in other environment not colab probably!')

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_file_in_drive_folder,
                             update_file_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder)
from src.constants import (
                           id_tracking_envios,
                           id_reporte_ventas,
                           id_business_case,
                           id_master_table_sheets,
                           id_clientes_apartado_sheets,
                           id_resultados_squad,
                           id_business_case_processed,
                           id_medios_total_sheets
                           )


warnings.filterwarnings('ignore')



In [ ]:
if from_drive:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import gspread
    from google.auth import default
    from gspread_dataframe import set_with_dataframe
    import gdown
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    ######################################################### get ids per file ###############################################################
    folder_id = '1hl7hITBmAXZKkb_qAHZ-045yJnSwfU2Q'

    file_id_dict = list_file_ids_for_drive_folder(drive,folder_id)

    id_dict = {}
    for k in file_id_dict.keys():
        id_dict[k] = {'id':file_id_dict.get(k),
                      'local_name':(k.strip()
                                    .lower()
                                    .replace(" - ", "_")
                                    .replace(" ", "_")
                                    .replace("-", "_"))
                    }
    print(f'Nombres archivos locales: {[id_dict[k]['local_name'] for k in id_dict ]}')
    ##################################################### get all files to local filesystem ###################################################
    for k in id_dict.keys():
        from_drive_to_local(drive, id_dict.get(k).get('id'), id_dict.get(k).get('local_name'))
    ###########################################################################################################################################
    data_date = get_last_modification_date_drive(drive,file_id_dict['Data Warehouse - Productivo.xlsx'])
    ###########################################################################################################################################
    creds, _ = default()
    gc = gspread.authorize(creds)

In [ ]:
ra = RawAtlas()